<a href="https://colab.research.google.com/github/kristina-skoptsova/image_processing/blob/main/image2vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Пример векторизации изображений


## Упрощенно по полутонам


In [1]:
!git clone 'https://github.com/kristina-skoptsova/image_processing'

Cloning into 'image_processing'...
remote: Enumerating objects: 2330, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 2330 (delta 27), reused 155 (delta 16), pack-reused 2152 (from 1)
Receiving objects: 100% (2330/2330), 131.46 MiB | 29.89 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [3]:
!pip install svgwrite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 3.0 MB/s eta 0:00:00


In [5]:
from PIL import Image
import svgwrite

# Функция, которая позволяет конвертировать растровое изображение в векторный формат 1 0

def raster_to_vector(input_file, output_file):
    # Открываем изображение и конвертируем в черно-белый формат
    image = Image.open(input_file).convert("L")
    bw_image = image.point(lambda x: 0 if x < 128 else 255, "1")

    # Получаем размер изображения
    width, height = bw_image.size

    # Создаем новый SVG файл
    dwg = svgwrite.Drawing(output_file, profile="tiny", size=(width, height))

    # Обходим все пиксели изображения и добавляем линии в SVG
    for y in range(height):
        for x in range(width):
            if bw_image.getpixel((x, y)) == 0:  # Черный пиксель
                # Рисуем квадрат для черного пикселя
                dwg.add(dwg.rect(insert=(x, y), size=(1, 1), fill="black"))

    # Сохраняем SVG файл
    dwg.save()


# Пример использования
raster_to_vector("/content/image_processing/data/raster.png", "output_image.svg")

## C распознаванием контуров


In [9]:
import cv2
import numpy as np
import svgwrite
from PIL import Image


def raster_to_vector_with_contours(input_file, output_file):
    # Открываем изображение и конвертируем в серый формат
    image = cv2.imread(input_file, cv2.IMREAD_GRAYSCALE)

    # Применяем алгоритм Кэнни для обнаружения краев
    edges = cv2.Canny(image, 50, 100) # 100 -  начальный уровень границ(игнор), 200 - иксели с градиентом выше этого значения будут определены как края(классификация,связь)
# чем меньше, тем больше четкость
    # Находим контуры на изображении
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Получаем размер изображения
    height, width = edges.shape

    # Создаем новый SVG файл
    dwg = svgwrite.Drawing(output_file, profile="tiny", size=(width, height))

    # Добавляем контуры в SVG файл
    for contour in contours:
        # Преобразуем контур в список кортежей с координатами
        points = [(int(point[0][0]), int(point[0][1])) for point in contour]
        if points:
            # Добавляем полилинию в SVG файл
            dwg.add(dwg.polyline(points, stroke="black", fill="none"))

    # Сохраняем SVG файл
    dwg.save()


# Пример использования
raster_to_vector_with_contours("/content/image_processing/data/raster.png", "output_image.svg")